In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers==4.3.3
!pip install stop_words
!pip install symspellpy
!pip install language_detector 
!pip install cached_property
!pip install sentencepiece
!pip install config
!pip install umap
!pip install sentence-transformers==0.4.1
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

     |████████████████████████████████| 1.9 MB 4.0 MB/s 
     |████████████████████████████████| 3.3 MB 45.3 MB/s 
     |████████████████████████████████| 895 kB 67.6 MB/s 
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32912 sha256=d075bae5f2f9255f9e596e8c42e75c73a08f810a524345771f9527b2a5b3213e
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words
     |████████████████████████████████| 2.6 MB 3.9 MB/s 
  Created wheel for language-detector: filename=language_detector-5.0.2-py3-none-any.whl size=7053 sha256=28df9903e1cdd5d3202e986d0f0534806c84da68ab2cdfd02a1fd307d0a5a96d
  Stored in directory: /root/.cache/pip/wheels/12/40/73/a0765d65e793332b79dfe6c34c713e7c0066ea785191b3f50a
Successfully built language-detector
     |████████████████████████████████| 1.2 MB 4.2 MB/s 
  Created wheel for umap: filename=umap-0.1.1-py3-none-any.whl size=3564 sha256=bef8e11c9d2482bfdb2

True

In [ ]:
root_path = "/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2"
import sys, os
import config

config.root_path = os.path.abspath(root_path)
sys.path.insert(0, config.root_path)

In [ ]:
import tensorflow as tf

from tensorflow.python import keras
import json
import pandas as pd
import numpy as np
import os

from src.encoders.context_encoder_ldabert_2 import ContextEncoderComplex, ContextEncoderSimple
from src.dataset.ldabert_2 import LDABERT2Dataset
import transformers
import sentence_transformers

from utils.metrics import windowdiff, pk

In [ ]:
tf.__version__, transformers.__version__, sentence_transformers.__version__

('2.6.0', '4.3.3', '0.4.1')

## LDA BERT 2 Dataset

In [ ]:
dataset_type = "clinical"
dataset = LDABERT2Dataset(dataset_type=dataset_type,
                           pct_data=0.075,
                          max_seq_length=128,
                           max_segment_length=300,
                           augment_pct=0)

sentences, tokenized_sentences, labels = dataset.process()

# vectors_path = '../data/clinical_vectors/lda_bert_{}_{}.pkl'.format(dataset_type, len(sentences))
vectors_filename = '{}_{}.pkl'.format(dataset.pct_data, dataset.augment_pct)

saved_vectors, saved_labels, saved_tokens, saved_tokenized_sentences = dataset.get_saved_vectors("test", dataset.dataset_type, vectors_filename)

if len(saved_vectors) == 0:
    saved_vectors, saved_labels, saved_tokens, saved_tokenized_sentences = dataset.create_vectors("test", dataset.dataset_type, vectors_filename)

artificial segments False
artificial segments False
something went wrong [Errno 2] No such file or directory: '/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2/data/lda_bert_2/generated_vectors/test/fiction/0.075_0.pkl'
root path /content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2
Preprocessing raw texts ...
sentences length 4510
Preprocessing raw texts. Done!
lda sentences length 4510
Getting vector representations for LDA ...
Getting vector representations for LDA. Done!
saving vectors... 4510 4510 4510


In [ ]:
left_input, mid_input, right_input = dataset.format_sentences_tri_input_plus(saved_tokenized_sentences)

In [ ]:
lda_left_input, lda_mid_input, lda_right_input = dataset.format_sentences_tri_input(saved_vectors)

In [ ]:
saved_tokenized_sentences['input_ids'].shape, saved_vectors.shape

(TensorShape([4510, 128]), (4510, 12))

In [ ]:
saved_tokens

[' FORGIVENESS',
 '  She had been brought up in one of those families who live entirely to themselves, apart from all the rest of the world. Such families know nothing of political events, although they are discussed at table; for changes in the Government take place at such a distance from them that they are spoken of as one speaks of a historical event, such as the death of Louis XVI or the landing of Napoleon. Customs are modified in course of time, fashions succeed one another, but such variations are taken no account of in the placid family circle where traditional usages prevail year after year. And if some scandalous episode or other occurs in the neighborhood, the disreputable story dies a natural death when it reaches the threshold of the house. The father and mother may, perhaps, exchange a few words on the subject when alone together some evening, but they speak in hushed tones—for even walls have ears. The father says, with bated breath:',
 '  "You\'ve heard of that terribl

## Model

In [ ]:
lda_gamma = 15
# model = ContextEncoderComplex(final_dropout=0.5,
#                             dense_neurons=256,
#                                   lstm_size=256,
#                              lstm_dropout_percentage=0.5,
#                              cnn_filters=8,
#                              cnn_kernel_size=3,
#                              pool_size=2,
#                           gamma=lda_gamma,
#                           max_sentence_length=128,
#                            bert_trainable=False)
model = ContextEncoderSimple(final_dropout=0.5,
                            dense_neurons=256,
                          gamma=lda_gamma,
                          max_sentence_length=128,
                           bert_trainable=False)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
hash = "BfCoW"
num_samples = "57488"
dataset_type = "fiction"
model_path = 'LDABERT2/complex/{}-{}-1-pct-1-aug_{}/no-finetune'.format(dataset_type, num_samples, hash)
checkpoint_filepath = '../models/{}/checkpoint'.format(model_path)
log_file = '../predictions/{}_lda-{}/predictions.csv'.format(hash, lda_gamma)

In [ ]:
# try:
model.load_weights(checkpoint_filepath)
# except:
#     print("No checkpoint available.")

In [ ]:
saved_tokenized_sentences

{'input_ids': <tf.Tensor: shape=(4510, 128), dtype=int32, numpy=
array([[  101, 17213,   102, ...,     0,     0,     0],
       [  101,  2016,  2018, ...,  2792,  2030,   102],
       [  101,  1000,  2017, ...,     0,     0,     0],
       ...,
       [  101,  1037,  9384, ...,     0,     0,     0],
       [  101,  2057,  2031, ...,     0,     0,     0],
       [  101,  1998,  2000, ...,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(4510, 128), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(4510, 128), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=in

## Inference

In [ ]:
slice_size = 125
predictions_log = []

for threshold in [0.5]:
  for i in range(0, saved_vectors.shape[0]//slice_size):
      start = i*slice_size
      end = start + slice_size
      print(start, end)

      sliced_tokenized_sentences = {
          'input_ids': saved_tokenized_sentences['input_ids'][start:end],
          'token_type_ids': saved_tokenized_sentences['token_type_ids'][start:end],
          'attention_mask': saved_tokenized_sentences['attention_mask'][start:end]
      }
      sliced_vectors = saved_vectors[start:end]
      
      left_input, mid_input, right_input = dataset.format_sentences_tri_input_plus(sliced_tokenized_sentences)
      lda_left_input, lda_mid_input, lda_right_input = dataset.format_sentences_tri_input(sliced_vectors)
      
      logits = model([
          left_input, 
          mid_input, 
          right_input, 
          lda_left_input, 
          lda_mid_input, 
          lda_right_input
      ])

      logits_flattened = [float(p) for p in logits]

      predictions = [1 if float(logit) >= threshold else 0 for logit in logits_flattened]

      k = 14

      string_predictions = "".join([str(i) for i in predictions])
      string_ground_truth = "".join([str(i) for i in saved_labels[start:end]])
      print(string_predictions)
      print(string_ground_truth)
      overall_windowdiff = windowdiff(string_predictions, string_ground_truth, k)
      overall_pk = pk(string_predictions, string_ground_truth, k)

      for pred_idx, pred in enumerate(predictions):
        log = {
            "prediction": pred,
            "ground_truth": saved_labels[start:end][pred_idx],
            "text": saved_tokens[start:end][pred_idx],
            "lda_gamma": lda_gamma,
            "prediction_threshold": threshold,
            "logit": logits_flattened[pred_idx]
        }
        predictions_log.append(log)

      print("{},{},{},{}".format(overall_windowdiff, overall_pk, k, k))

0 125
10000000000000000000000000000011000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000
10000000000000000000000000000001000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000
0.125,0.008928571428571428,14,14
125 250
00000010001000000000000001000000001001000000010000000000000000000000000000000000000000010000110001000100000000000000000001110
00000010001000000000000001000000001001000000000000000000000000000000000000000000000000010000100000000000000000000000000000100
0.35714285714285715,0.16071428571428573,14,14
250 375
10000000000000000000100001000000000000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000
00000000000000000000100000000000000000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000
0.13392857142857142,0.05357142857142857,14,14
375 500
00000000000000000000000000000001000000000000000000000000000000000000000000001001100000000

In [ ]:
try:
  import csv
  if not os.path.exists(os.path.dirname(log_file)):
      os.makedirs(os.path.dirname(log_file))
  mode = 'a' if os.path.exists(log_file) else 'w'
  with open(log_file, mode) as csvfile:
      writer = csv.DictWriter(csvfile, fieldnames=["prediction", "ground_truth", "text", "lda_gamma", "prediction_threshold", "logit"])
      writer.writeheader()
      for data in predictions_log:
          writer.writerow(data)
except IOError:
    print("I/O error")

In [ ]:
[float(p) for p in predictions], saved_labels

([0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0],
 [1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,